# How did life expectancy and fertility change between 1964 - 2013 for different countries and continents?
## What region developed the most in this time period?

Annette Brak <br> 12146218 <br> 14-12-2022

# Data
### Loading in data

In [1]:
from os.path import dirname, join
import pandas as pd
import geopandas as gpd

Loading the given Gapminder data

In [2]:
# load file (for bokeh server)
gapminder_load = pd.read_csv(join(dirname(__file__), 'data/gapminder.csv'))

In [3]:
# select only columns of interest
df_gapminder = gapminder_load[['ID', 'Country', 'Year', 'lifeExp', 'Fertility', 'Region']].copy()
df_gapminder.rename(columns = {'lifeExp':'Life_Exp'}, inplace = True)

Loading geometry data from [Our Natural Earth](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-countries/) for plotting a world map

In [4]:
# load file (for bokeh server)
gpd_load = gpd.read_file(join(dirname(__file__), 'data/countries_10m/ne_10m_admin_0_countries.shp'))

In [5]:
# Select only columns of interest
geo_data_raw = gpd_load[['SOV_A3', 'ADMIN', 'geometry']]
geo_data_raw.columns = ['ID', 'Country', 'geometry']

### Cleaning up data

The country names and IDs in both these datasets do not correspond. To make sure they do before merging, I used the [country converter](https://github.com/konstantinstadler/country_converter) package to easily convert these values in both datasets.

In [6]:
import country_converter as coco
import logging

By default, the country converter gives an error for all values where no match is found, but these issues are not critical for its performance and will be solved separately. These warnings can therefore be suppressed.

In [7]:
# set country converter warning to only show critical warnings 
coco_logger = coco.logging.getLogger()
coco_logger.setLevel(logging.CRITICAL)

#### Gapminder data

In [8]:
# replace country names with their official, shortened, names
df_gapminder['Country'] = coco.convert(names = df_gapminder['Country'], to = 'name_short', not_found = None)

# replace country IDs with their official ISO alpha-2 codes
df_gapminder['ID'] = coco.convert(names = df_gapminder['Country'], to = 'ISO2', not_found = None)

Two countries are cannot be matched: the Channel Islands (consisting of separate countries beloning to the UK) and the Netherlands Antilles (broken up into individual countries in 2010). The Channel Islands have no official ISO code anymore, but their previous ISO alpha-3 code was GB-CHA. The Netherlands Antilles used to have ISO alpha-2 code AN.

In [9]:
# convert Channel islands
cha_mask = df_gapminder['ID'] == 'Channel Islands'
df_gapminder.loc[cha_mask, 'ID'] = 'GB-CHA'

In [10]:
# convert antilles ID to AN
an_mask = df_gapminder['ID'] == 'Netherlands Antilles'
df_gapminder.loc[an_mask, 'ID'] = 'AN'

#### Geodata

Several French areas are labeled as country 'France' on the map. In our gapminder dataset, some of these areas are separately defined with their own data. Therefore, I separated the 'France' geometry data into these separately defined areas as their own countries.

In [11]:
# exploding the entry for france from one multipolygon into single polygons
fr_exploded = (geo_data_raw[geo_data_raw['Country'] == 'France'].explode(ignore_index = True))

Plotting these areas on a map and comparing them to Google Maps allowed me to see which rows corresponded to which area(s). This did mean that I had to hard code these areas, as I could not find an efficient way to soft code this part.

In [12]:
# renaming the indices accordingly
fr_exploded.iloc[0,0] = 'GF'
fr_exploded.iloc[0,1] = 'French Guiana'

fr_exploded.iloc[2,0] = 'MQ'
fr_exploded.iloc[2,1] = 'Martinique'

fr_exploded.iloc[[3,4,5,6,18,19],0] = 'GP'
fr_exploded.iloc[[3,4,5,6,18,19],1] = 'Guadeloupe'

fr_exploded.iloc[7,0] = 'RE'
fr_exploded.iloc[7,1] = 'Reunion'

fr_exploded.iloc[9,0] = 'YT'
fr_exploded.iloc[9,1] = 'Mayotte'

In [13]:
# dissolving the data to recombine corresponding areas into multipolygons
fr_dissolved = fr_exploded.dissolve(by = 'Country', as_index = False)

The easiest way to replace the data in the original geo data is to remove the original entry for France and append our newly specified set of separated areas.

In [14]:
# removing 'France' entry from dataset
geo_data_raw = geo_data_raw.loc[~(geo_data_raw['Country'] == 'France')]

In [15]:
# append our newly defined entries for france and french areas
geo_data = geo_data_raw.append(fr_dissolved)

Antarctica takes up a lot of space on the map, but the Gapminder dataset has no recorded data on it. To make the map easier to read, I removed it from the map.

In [16]:
geo_data = geo_data.loc[~(geo_data_raw['ID'] == 'ATA')]

Now, to replace the country names and IDs so they match the Gapminder data.

In [17]:
# replace country names with their official, shortened, names
geo_data['Country'] = coco.convert(names = geo_data['Country'], to = 'name_short', not_found = None)

# replace country IDs with their official ISO alpha-2 codes
geo_data['ID'] = coco.convert(names = geo_data['Country'], to = 'ISO2', not_found = None)

In our Gapminder dataset, the Channel Islands and the Netherlands Antilles are taken together as one country, whereas on our map these are separated into different countries. To best display our recorded data, we will count these countries under the areas defined in the Gapminder dataset.

In [18]:
# convert Curacao and Sint Maarten back to Netherlands Antilles
sx_cw_mask = (geo_data['ID'] == 'SX') | (geo_data['ID'] == 'CW')
geo_data.loc[sx_cw_mask, 'ID'] = 'AN'
geo_data.loc[sx_cw_mask, 'Country'] = 'Netherlands Antilles'

In [19]:
# convert Jersey and and Guernsey back to Channel Islands
je_gg_mask = (geo_data['ID'] == 'JE') | (geo_data['ID'] == 'GG')
geo_data.loc[je_gg_mask, 'ID'] = 'GB-CHA'
geo_data.loc[je_gg_mask, 'Country'] = 'Channel Islands'

Note: not all regions in the geometry dataset are matched in the country conversion database. However, after these edits, all countries recorded in the Gapminder dataset are represented in the geometry dataset.

# Merging the data

In [20]:
# merging geo and gapminder data
df_full_raw = geo_data.merge(df_gapminder, on = 'ID', how = 'left')

As a result of the merge, two columns for country names are now in our dataset. One of these can be dropped, since country names for areas with recorded data coincide with those in the geometry data.

In [21]:
# drop last country column, rename leftover country column
df_full = df_full_raw.drop(columns = 'Country_y')
df_full = df_full.rename(columns = {'Country_x': 'Country'})

Some countries appear in the dataset twice, once with recorded data and once without. To combine these rows, I first fill NaN values in the Year column, so countries without any recorded data are not lost. Then, I grouped data by country and year, taking the first non-NaN values for each of these groups.

In [22]:
# fill NaN year values for countries without recorded data
df_full['Year'] = df_full['Year'].fillna('No data')

In [23]:
# group data, take first non-empty value if found
df_clean = df_full.groupby(['Country', 'Year']).first().reset_index()

# Preparing world map plot

In [52]:
import json
from bokeh.palettes import brewer, Bokeh
from bokeh.models import (GeoJSONDataSource, LinearColorMapper, ColorBar, FixedTicker, HoverTool, Slider, Legend, Tabs, TabPanel)

In the world map, you will be able to filter the data per year. To continuously update, I made a function for filtering the data. I also created a slider tool to select the preferred year(s) with.

In [53]:
def filter_data(df, yr):
    """ Filters entries given (geo)dataframe in a given year, returns json of filtered data"""
    year_mask = df['Year'].isin(['No data', yr])
    df_year = df[year_mask]
    df_filtered = df_year.to_json()
    return df_filtered

df_2013 = filter_data(df_clean, 2013)

In [54]:
# create slider for filtering by year
slider = Slider(title = 'Year', start = 1964, end = 2013, step = 1, value = 2013)

The color palettes for both the Fertility and the Life Expectancy maps are easier to create before plotting, just like the Hover tool for both maps.

In [55]:
# load reversed brewer palette for both maps
F_palette = brewer['RdPu'][9][::-1]
LE_palette = brewer['GnBu'][9][::-1]

# create hover tool to be used for both world maps
F_hover = HoverTool(tooltips = [('Country', '@Country (@ID)'), ('Fertility rate', '@Fertility')])
LE_hover = HoverTool(tooltips = [('Country', '@Country (@ID)'), ('Life Expectancy', '@Life_Exp')])

By default, the world map will be plotted for 2013.

In [56]:
# input source for map plotting
geo_source = GeoJSONDataSource(geojson = df_2013)

# Plotting the world map

In [57]:
def plot_map(statistic, low, high, step):
    
    if statistic == 'F':
        palette = F_palette
        hover = F_hover
        name = 'fertility rate'
        col = 'Fertility'
        
    elif statistic == 'LE':
        palette = LE_palette
        hover = LE_hover
        name = 'life expectancy'
        col = 'Life_Exp'
    
    # define color map for plot
    color_map = LinearColorMapper(palette = palette, low = low, high = high, nan_color = '#d9d9d9')
    
    # set ticks for color bar based on data range
    ticks = list(range(low, high+1, step))
    color_ticks = FixedTicker(ticks=ticks)
    
    # create world map figure
    p_map = figure(title = f'Global {name}', width = 750, height = 450, x_range = (-182,182), y_range = (-60, 85),
               toolbar_location = 'right', tools = 'pan, wheel_zoom, reset', active_scroll = 'wheel_zoom')

    p_map.xgrid.grid_line_color = None
    p_map.ygrid.grid_line_color = None
    p_map.axis.visible = False
    
    # plot countries as patches
    patches = p_map.patches('xs','ys', source = geo_source, fill_color = {'field': col, 'transform': color_map},
                        line_color = 'black', line_width = 0.25, fill_alpha = 1, hover_line_width = 3, hover_alpha = 0.75)

    # create color bar 
    color_bar = ColorBar(color_mapper = color_map, width = 500, height = 10, orientation = 'horizontal', location = 'center',
                         ticker = color_ticks, major_label_overrides = {40: '<40'})
    
    p_map.add_layout(color_bar, 'below')
    
    # add hover tool
    p_map.add_tools(hover)

    return p_map

Interactivity will be done using a Bokeh server. Using a python function, the map will be updated for the year selected by using the slider.

In [58]:
# update world map input
def update_year(attr, old, new):
    """Re-plots world map for selected year"""
    yr = slider.value
    new_data = filter_data(df_clean, yr)
    geo_source.geojson = new_data

# inputs to above function when slider value changes
slider.on_change('value_throttled', update_year)

# Preparing regional graph plots

I will also plot the average changes in both Fertility and Life Expectancy per region, and on a global level. For this, I first created a new dataset with only regional averages per year.

In [59]:
# get average values for life expectancy and fertility per region
df_regions = df_clean.groupby(['Region', 'Year'], as_index = False)[['Life_Exp', 'Fertility']].mean()

# calculate global averages by averaging all regions
global_means = df_regions.groupby('Year', as_index = False)[['Life_Exp', 'Fertility']].mean()

# region column gets lost in last step: re-add column
global_means['Region'] = 'Global'

# combine both regional and global data into one dataset
total_growth = df_regions.append(global_means)

I also defined a new function to easily calculate the differences between 2013 and 1964 for each region.

In [60]:
def calculate_growth(region, col):
    """Calculates numerical difference in value between first measured year (1964) and last measured year (2013)
    for any given region and numerical column"""
    region_mask = total_growth['Region'] == region
    selected_region = total_growth[region_mask]
    
    first_mask = selected_region['Year'] == 1964
    last_mask = selected_region['Year'] == 2013
    
    starting_value = selected_region.loc[first_mask, col].item()
    end_value = selected_region.loc[last_mask, col].item()
    growth = end_value - starting_value
    
    return round(growth, 2)

# Plot regional graphs

In [61]:
def plot_graph(statistic):
    
    if statistic == 'F':
        col = 'Fertility'
        name = 'Fertility rate'
        legend_loc = 'left'
        y_range = (0, 9)
        change_icon = '↓'
        
    elif statistic == 'LE':
        y_range = (40, 80)
        col = 'Life_Exp'
        name = 'Life expectancy'
        legend_loc = 'right'
        change_icon = '↑'
    
    # set up Hover tool to be used in figure
    hover_setup = HoverTool(tooltips = [('Year', '@x'), (name, '@y')])
    
    # import reverse Bokeh palette
    palette = Bokeh[6][::-1]
    
    # get list of unique regions in dataset
    region_list = total_growth['Region'].unique()
    
    # get all years in dataset for x axis
    xs = total_growth['Year'].unique()
    
    # create figure graphs will be plotted in
    p_graph = figure(title = f'{name} from 1964 - 2013', height = 450, width = 750, x_range = (1964, 2013),
                     toolbar_location = 'above', tools = 'pan, wheel_zoom, reset', active_scroll = 'wheel_zoom')
    
    p_graph.xaxis.axis_label = 'Year'
    p_graph.yaxis.axis_label = name
    
    # add legend layout before plotting lines
    # note: allows legend to be outside of plot
    p_graph.add_layout(Legend(), legend_loc)
    
    # loop over regions for plotting
    for region, i in zip(region_list, range(0, len(region_list))):
        
        # global data is to be displayed differently than regional data
        if region == 'Global':
            color = 'black'
            dash = 'dashed'
            
        else:
            color = palette[i]
            dash = 'solid'
        
        # calculate overall change in measured value, store result in label > will be displayed in legend
        growth = calculate_growth(region, col)
        label_with_growth = f'{region}\t({growth} {change_icon})'
        
        # get data values for given region and column
        region_mask = total_growth['Region'] == region
        y = total_growth.loc[region_mask, col]
        
        # plot line, keep line invisible in plot
        line = p_graph.line(xs, y, legend_label = label_with_growth, color = color, line_width = 3, line_dash = dash)
        line.visible = False
        
        # plot circles for hovering purposes, keep circles invisible in plot
        circle = p_graph.circle(xs, y, size = 15, fill_color = 'white', legend_label = label_with_growth, line_color = None,
                       fill_alpha = 0, hover_line_color = color, hover_fill_color = color, hover_alpha = 1)
        circle.visible = False
    
    # hide lines by clicking on legend iteration
    p_graph.legend.click_policy = 'hide'
    
    # add hover tool to figure
    p_graph.add_tools(hover_setup)

    return p_graph

# Creating final layout

In [66]:
from bokeh.layouts import gridplot, row
from bokeh.plotting import figure, curdoc

I will be creating a layout with two tabs: on the first tab world maps for both Fertility rates and Life Expectancy, on the second tab two graphs showing regional (and global) developments in Fertility rates and Life Expectancy over the years.

In [62]:
# plot world maps
F_plot = plot_map('F', 0, 9, 1)
LE_plot = plot_map('LE', 40, 85, 5)

# plot regional graphs
F_graph = plot_graph('F')
LE_graph = plot_graph('LE')

In [63]:
# create first tab for both world maps
tab_map = TabPanel(title = "World Map",
               child = gridplot([[F_plot, LE_plot], [slider]]))

# create second tab for both regional graphs
tab_graph = TabPanel(title = "Regional Development", child = row(F_graph, LE_graph))

# store both tabs in final layout
final_layout = Tabs(tabs = [tab_map, tab_graph])

Finally, the layout should be rendered in an active Bokeh server for the interactive slider to work.

In [67]:
# call plot in interactive Bokeh server
curdoc().add_root(final_layout)
curdoc().title = 'Fertility and Life Expectancy 1964 - 2013'

RuntimeError: Models must be owned by only a single document, ColumnDataSource(id='p10205', ...) is already in a doc